In [5]:
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
import numpy as np
from scipy.stats import chi2_contingency,chisquare
import pandas as pd
import copy, pickle
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import IsolationForest
from sklearn import preprocessing
from keras.models import Sequential
from keras.layers import Dense
from keras.models import model_from_json
from keras.optimizers import SGD

In [6]:
#reading data data set from excel
df=pd.read_excel('data\\local files\\AI EU Data.xlsx')

In [9]:
#identify datatype missing values
df_csv=pd.DataFrame([df.dtypes
                             ,df.isna().sum()
                             ,df.count()
                             ,(df.isna().sum()/df.count().max())*100]
                           ,index = ['Datatype','Missing_count','Availability_count','Missing_percent']).T

In [10]:
df.head()

,Line of Business,BusinessClassification,InsuredState,InceptionDate,ExpiryDate,IsAdmitted,Type,status,Lead Excess-All Other_c,Lead Excess-Real Estate/Property Mgmt/Office Building_c,...,Total Layer Limit_l,General Liability Allocation_l,Coverage Premium_l,Uninsured/Under Insured Motorist Premium_l,Attachment Amount_l,Per Occurrence limit_l,Layer Aggregate_l,Terrorism_l,Aggregate_l,Auto Liability Allocation_l
0,Excess & Umbrella,NaN,NJ,2015-07-01,2016-07-01,1,Renewal,Issued,0,0,...,6000000.0,1866.00,19658.0,NaN,15000000.0,6000000.0,6000000.0,3034.0,6000000.0,17724.00
1,Excess & Umbrella,NaN,IN,2015-06-30,2016-06-30,1,New,Issued,0,0,...,10000000.0,53512.00,60500.0,NaN,1000000.0,10000000.0,10000000.0,NaN,10000000.0,6988.00
2,Excess & Umbrella,NaN,MN,2015-04-01,2016-06-01,1,Renewal,Issued,0,0,...,50000000.0,65266.62,66519.0,NaN,50000000.0,50000000.0,50000000.0,NaN,50000000.0,1252.38
3,Excess & Umbrella,NaN,OR,2015-03-25,2016-07-01,1,New,Issued,0,0,...,2000000.0,3220.00,3220.0,NaN,2000000.0,2000000.0,2000000.0,NaN,2000000.0,NaN
4,Excess & Umbrella,NaN,CA,2015-05-31,2016-05-31,1,Renewal,Issued,0,0,...,25000000.0,68120.00,68750.0,NaN,25000000.0,25000000.0,25000000.0,3437.5,25000000.0,630.00


In [11]:
df_csv.head()

,Datatype,Missing_count,Availability_count,Missing_percent
Line of Business,object,0,36537,0
BusinessClassification,object,11663,24874,31.9211
InsuredState,object,16,36521,0.0437912
InceptionDate,datetime64[ns],0,36537,0
ExpiryDate,datetime64[ns],0,36537,0


In [12]:
#Column Datatype
DATATYPE = {
           'Line of Business'          :        'categorical',
           'BusinessClassification'    :        'categorical',
           'InsuredState'              :        'categorical',
           'InceptionDate'             :        'datetime',
           'ExpiryDate'                :        'datetime',
           'IsAdmitted'                :        'categorical',
           'Type'                      :        'categorical',
           'status'                    :        'categorical',
           'Lead Excess-All Other_c'   :        'categorical',
           'Lead Excess-Real Estate/Property Mgmt/Office Building_c':        'categorical',
           'Lead Excess-Contracting/Construction_c'                 :        'categorical',
           'Excess-Real Estate/Property Mgmt/Office Building_c'     :        'categorical',
           'Excess-Manufacturing_c'                                 :        'categorical', 
           'Lead Excess-Retail/Wholesale_c'                         :        'categorical',
           'Lead Excess-Municipalities/Hospitals/Political_c'       :        'categorical',
           'Lead Excess-Installation/Service or Repair_c'           :        'categorical',
           'Excess-Installation/Service or Repair_c'                :        'categorical',
           'Excess-Entertainment/Recreation_c'                      :        'categorical',
           'Excess-Retail/Wholesale_c'                              :        'categorical',
           'Excess-Entertainment/Recreation_l'                      :        'numerical',
           'Excess-Municipalities/Hospitals/Political_c'            :        'categorical',
           'Lead Excess-Manufacturing_c'                            :        'categorical',
           'Excess-Manufacturing_l'                                 :        'numerical',
           'Excess-Contracting/Construction_c'                      :        'categorical',
           'Lead Excess-All Other_l'                                :        'numerical',
           'Lead Excess-Real Estate/Property Mgmt/Office Building_l':        'numerical',
           'Excess-All Other_c'                                     :        'categorical',
           'Umbrella-Contracting/Construction'                      :        'categorical',
           'Lead Excess-Entertainment/Recreation_c'                 :        'categorical',
           'Excess-Contracting/Construction_l'                      :        'numerical',
           'Excess-Installation/Service or Repair_l'                :        'numerical',
           'Lead Excess-Installation/Service or Repair_l'           :        'numerical',
           'Excess-Real Estate/Property Mgmt/Office Building_l'     :        'numerical',
           'Excess-All Other_l'                                     :        'numerical', 
           'Excess-Retail/Wholesale_l'                              :        'numerical', 
           'Umbrella-All Other'                                     :        'categorical',
           'Lead Excess-Contracting/Construction_l'                 :        'numerical',
           'Lead Excess-Entertainment/Recreation_l'                 :        'numerical',
           'Lead Excess-Municipalities/Hospitals/Political_l'       :        'numerical',
           'Lead Excess-Manufacturing_l'                            :        'numerical',
           'Excess-Municipalities/Hospitals/Political_l'            :        'numerical',
           'Lead Excess-Retail/Wholesale_l'                         :        'numerical', 
           'Total Layer Limit_c'                                    :        'categorical',
           'General Liability Allocation_c'                         :        'categorical', 
           'Coverage Premium_c'                                     :        'categorical',
           'Uninsured/Under Insured Motorist Premium_c'             :        'categorical', 
           'Attachment Amount_c'                                    :        'categorical',
           'Per Occurrence limit_c'                                 :        'categorical', 
           'Layer Aggregate_c'                                      :        'categorical', 
           'Terrorism_c'                                            :        'categorical', 
           'Aggregate_c'                                            :        'categorical',
           'Auto Liability Allocation_c'                            :        'categorical', 
           'Total Layer Limit_l'                                    :        'numerical',
           'General Liability Allocation_l'                         :        'numerical', 
           'Coverage Premium_l'                                     :        'numerical',
           'Uninsured/Under Insured Motorist Premium_l'             :        'numerical', 
           'Attachment Amount_l'                                    :        'numerical',
           'Per Occurrence limit_l'                                 :        'numerical', 
           'Layer Aggregate_l'                                      :        'numerical', 
           'Terrorism_l'                                            :        'numerical',
           'Aggregate_l'                                            :        'numerical', 
           'Auto Liability Allocation_l'                            :        'numerical'
           }

In [47]:
#spliting datatype json data       
def split_data_on_datatypes(data_set,datatypes_json):
    data = {}
    hash_data=set(list(datatypes_json.values()))
    for dt in hash_data:        
        lst = []
        for k in datatypes_json.keys():
            if datatypes_json[k] == dt:
                lst.append(k)
        data[dt] = lst
    for key in data.keys():        
        data[key] = data_set[data[key]]
        
    return data

In [48]:
datatypes_json_data=DATATYPE

split_data = split_data_on_datatypes(df,datatypes_json_data)
#set(datatypes_json.values()

In [49]:
split_data

{'numerical':        Excess-Entertainment/Recreation_l  Excess-Manufacturing_l  \
 0                                      0                       0   
 1                                      0                       0   
 2                                      0                       0   
 3                                      0                       0   
 4                                      0                       0   
 5                                      0                       0   
 6                                      0                       0   
 7                                      0                       0   
 8                                      0                       0   
 9                                      0                       0   
 10                                     0                       0   
 11                                     0                       0   
 12                                     0                       0   
 13                  

In [51]:
def impute(x,y):
    if (x is None) or (x != x):
        return y
    else:
        return x 

In [52]:
for each in split_data['numerical']:
    y = 0
    split_data['numerical'][each] = split_data['numerical'][each].apply(lambda x: impute(x,y))
        
for each in split_data['categorical']:
    split_data['categorical'][each] = split_data['categorical'][each].apply(lambda x: impute(x,'unknown'))

features = ['Layer Aggregate_l', 'Total Layer Limit_l', 'Per Occurrence limit_l','Aggregate_l']


C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [53]:
split_data

{'numerical':        Excess-Entertainment/Recreation_l  Excess-Manufacturing_l  \
 0                                      0                       0   
 1                                      0                       0   
 2                                      0                       0   
 3                                      0                       0   
 4                                      0                       0   
 5                                      0                       0   
 6                                      0                       0   
 7                                      0                       0   
 8                                      0                       0   
 9                                      0                       0   
 10                                     0                       0   
 11                                     0                       0   
 12                                     0                       0   
 13                  

In [57]:
# Separating out the features
df_num = split_data['numerical']
x = df_num[features]

In [60]:
 # Standardizing the features
std_scaler = StandardScaler().fit(x)

pickle.dump(std_scaler, open( r'C:\Users\sunil007\Documents\MachineLearning\PremiumAdvice\pickle\encoder\std_scaler.pkl', "wb" ) )

In [61]:
std_scale = pickle.load( open( r'C:\Users\sunil007\Documents\MachineLearning\PremiumAdvice\pickle\encoder\std_scaler.pkl', "rb" ) )
x = std_scale.transform(x)


In [62]:
x

array([[-0.00525418, -0.00525223, -0.00522434, -0.00513758],
       [-0.00510126, -0.00509931, -0.00507142, -0.00498466],
       [-0.00357207, -0.00357012, -0.00354222, -0.00345547],
       ...,
       [-0.00513949, -0.00513754, -0.00510965, -0.00536696],
       [-0.00544533, -0.00544338, -0.00541549, -0.00532873],
       [-0.00529241, -0.00529046, -0.00526257, -0.00517581]])

In [64]:
 # PCA
    
pca = PCA(n_components=1)
principalComponents = pca.fit(x)
pickle.dump( principalComponents, open( r'C:\Users\sunil007\Documents\MachineLearning\PremiumAdvice\pickle\encoder\pca.pkl', "wb") )

In [65]:
pca.components_

array([[0.5, 0.5, 0.5, 0.5]])

In [66]:
x.shape

(36537, 4)

In [67]:
principalComponent = pickle.load( open( r'C:\Users\sunil007\Documents\MachineLearning\PremiumAdvice\pickle\encoder\pca.pkl', "rb" ) )
principalComponents = principalComponent.transform(x)
principalDf = pd.DataFrame(data = principalComponents, columns = ['principal component 1'])
    

In [68]:
principalComponents.shape

(36537, 1)

In [70]:
principalDf.head()

,principal component 1
0,-0.010434
1,-0.010128
2,-0.007070
3,-0.010740
4,-0.008981


In [71]:
# merging PCA and other numerical columns
df_num.drop(features, axis=1, inplace=True)
num_data = pd.concat([principalDf,df_num],axis=1)


C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\frame.py:3694: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)


In [72]:
num_data.head()

,principal component 1,Excess-Entertainment/Recreation_l,Excess-Manufacturing_l,Lead Excess-All Other_l,Lead Excess-Real Estate/Property Mgmt/Office Building_l,Excess-Contracting/Construction_l,Excess-Installation/Service or Repair_l,Lead Excess-Installation/Service or Repair_l,Excess-Real Estate/Property Mgmt/Office Building_l,Excess-All Other_l,...,Lead Excess-Municipalities/Hospitals/Political_l,Lead Excess-Manufacturing_l,Excess-Municipalities/Hospitals/Political_l,Lead Excess-Retail/Wholesale_l,General Liability Allocation_l,Coverage Premium_l,Uninsured/Under Insured Motorist Premium_l,Attachment Amount_l,Terrorism_l,Auto Liability Allocation_l
0,-0.010434,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1866.00,19658.0,0.0,15000000.0,3034.0,17724.00
1,-0.010128,0,0,0,0,0,0,0,0,0,...,0,0,0,0,53512.00,60500.0,0.0,1000000.0,0.0,6988.00
2,-0.007070,0,0,0,0,0,0,0,0,0,...,0,0,0,0,65266.62,66519.0,0.0,50000000.0,0.0,1252.38
3,-0.010740,0,0,0,0,0,0,0,0,0,...,0,0,0,0,3220.00,3220.0,0.0,2000000.0,0.0,0.00
4,-0.008981,0,0,0,0,0,0,0,0,0,...,0,0,0,0,68120.00,68750.0,0.0,25000000.0,3437.5,630.00


In [73]:
y_cat =  split_data['categorical']['status']

## label encoding categorical variable
dt =pd.DataFrame()
for each in split_data['categorical'].columns:
    label_encoder = LabelEncoder()
    enc = label_encoder.fit(split_data['categorical'][each])
    pickle.dump( enc, open( r'C:\Users\sunil007\Documents\MachineLearning\PremiumAdvice\pickle\encoder\encoder_' +  str(each.replace('/','_')) +'.pkl', "wb" ) )
    

In [74]:
for each in split_data['categorical'].columns:    
    encoder = pickle.load( open( r'C:\Users\sunil007\Documents\MachineLearning\PremiumAdvice\pickle\encoder\encoder_' +  str(each.replace('/','_')) +'.pkl', "rb" ) )
    enc = encoder.transform(split_data['categorical'][each])
    dt = pd.concat([dt,pd.DataFrame(enc)],axis=1)
 

In [75]:
dt.columns = split_data['categorical'].columns
y_num = dt['status']
dt.drop('Line of Business',axis = 1,inplace = True) # removing Line of Business because there is only one Line of Business


In [77]:
dt.head()

,Line of Business,BusinessClassification,InsuredState,IsAdmitted,Type,status,Lead Excess-All Other_c,Lead Excess-Real Estate/Property Mgmt/Office Building_c,Lead Excess-Contracting/Construction_c,Excess-Real Estate/Property Mgmt/Office Building_c,...,Total Layer Limit_c,General Liability Allocation_c,Coverage Premium_c,Uninsured/Under Insured Motorist Premium_c,Attachment Amount_c,Per Occurrence limit_c,Layer Aggregate_c,Terrorism_c,Aggregate_c,Auto Liability Allocation_c
0,0,864,33,1,1,7,0,0,0,0,...,1,1,1,0,1,1,1,1,1,1
1,0,864,17,1,0,7,0,0,0,0,...,1,1,1,0,1,1,1,0,1,1
2,0,864,25,1,1,7,0,0,0,0,...,2,2,2,0,2,2,2,0,2,2
3,0,864,39,1,0,7,0,0,0,0,...,2,2,2,0,2,2,2,0,2,0
4,0,864,6,1,1,7,0,0,0,0,...,1,1,1,0,1,1,1,1,1,1


In [78]:
y_num.head()

0    7
1    7
2    7
3    7
4    7
Name: status, dtype: int64